In [11]:
import json
from collections import Counter, defaultdict
import sys
import utils
import pickle
import pandas as pd

In [41]:
input_path='../data/input'
gva_input_annotations_file = '%s/annotation/participants_input.p' % input_path
gva_samefirstname_annotations = '%s/annotation/participants_samefirstname.p' % input_path
gva_samelastname_annotations = '%s/annotation/participants_samelastname.p' % input_path

aux_path='auxiliary_data'
str_data_path='%s/str_data.json' % aux_path
doc2inc_file='%s/doc2inc.p' % aux_path

### 1. Find participants with multiple incidents

The original purpose of this was to find good examples for participants where the profiling could help establishing identity.

In [3]:
names_set=utils.get_all_names_set(gva_input_annotations_file)
len(names_set)

456

In [4]:
names_list=utils.get_all_names_list(gva_input_annotations_file)
len(names_list)
cntr=Counter(names_list).most_common()

In [5]:
with open (str_data_path) as f:
    str_data=json.load(f)

In [6]:
part_incidents_pairs=0
for participant_name, freq in cntr:
    participant_name=participant_name.strip()
    incs=set()
    for inc, parts in str_data.items():
        for part in parts:
            if 'Name' in part and part['Name'].strip()==participant_name:
                incs.add(inc)
    if len(incs)>1:
        print(participant_name, incs)
    part_incidents_pairs+=len(incs)

Vincent Wainwright {'409732', '479709'}
Henry Jackson {'533821', '97407'}
Carl Johnson {'321156', '448419'}
Quanisha Sims {'558601', '558618'}
Omar Hamilton {'767876', '296327'}
Daniel Munoz {'494072', '384047'}
Allen Troyer {'389279', '502850'}
Saeve Evans {'710200', '172205'}
Tyquone Greer {'113247', '113214'}
Ismael Moreno Jr. {'152722', '156813'}
Ronald Williams {'481229', '510769', '387194', '588835'}
Rakim Watson {'510769', '588835', '317165'}
Lawrence Anderson {'735589', '732572'}


In [7]:
part_incidents_pairs

472

### [TODO] Fix the data, by: merging the same participants with different names, and by adding BU data

### 2. Compute ambiguity for the various datasets 

In [8]:
print('Ambiguity:')
print(part_incidents_pairs/len(names_set))

Ambiguity:
1.0350877192982457


In [9]:
names_set_samefirst=utils.get_all_names_set(gva_samefirstname_annotations)
len(names_set_samefirst)

325

In [10]:
print(part_incidents_pairs/len(names_set_samefirst))

1.4523076923076923


In [11]:
names_set_samelast=utils.get_all_names_set(gva_samelastname_annotations)
len(names_set_samelast)

377

In [12]:
print(part_incidents_pairs/len(names_set_samelast))

1.2519893899204244


### 3. Inspect GVDB

In [13]:
import pandas as pd
import json

In [14]:
gvdb_articles_path = "../../gvdb-aggregated-db/Articles-with-extracted-info.tsv"

In [15]:
#with open(gvdb_articles_path) as f:
gvdb_articles = pd.read_csv(gvdb_articles_path, sep='\t')
#    csv.reader(f, delimiter='\t', quotechar='"')
#    print(len(lines))


In [16]:
len(gvdb_articles)

23903

In [17]:
sections=['shooter-section', 'victim-section']

In [18]:
cnt_attr_values=0
attr='gender'
values=[]
states=[]
dates=[]
for index, row in gvdb_articles.iterrows():
    annotations=json.loads(row['Json'])
    if annotations['date-and-time']['state']:
        states.append(annotations['date-and-time']['state'].strip())
    if annotations['date-and-time']['date']:
        dates.append(annotations['date-and-time']['date'].strip()[:4])
    
    #print(annotations)
    #break
    for sec in sections:
#        print(annotations[sec])
        for part in annotations[sec]:
            if part[attr]:#['value']:
                #print(part[attr]['value'])
                cnt_attr_values+=1
                values.append(part[attr].lower().strip())

In [19]:
cnt_attr_values

36574

In [20]:
for v in Counter(values).most_common():
    print(v[0],'\t', v[1])

male 	 31980
female 	 4594


In [21]:
for v in Counter(states).most_common():
    print(v[0],'\t', v[1])

CA - California 	 3514
Unclear 	 3337
CO - Colorado 	 3012
CT - Connecticut 	 1322
Colorado 	 1291
PA - Pennsylvania 	 960
Connecticut 	 659
IL - Illinois 	 610
TN - Tennessee 	 466
TX - Texas 	 452
FL - Florida 	 395
NY - New York 	 387
OH - Ohio 	 376
NJ - New Jersey 	 306
IN - Indiana 	 282
VA - Virginia 	 260
GA - Georgia 	 258
NC - North Carolina 	 252
WI - Wisconsin 	 229
LA - Louisiana 	 218
MI - Michigan 	 207
Outside the USA 	 202
SC - South Carolina 	 187
MO - Missouri 	 175
MN - Minnesota 	 169
AL - Alabama 	 157
MD - Maryland 	 153
AZ - Arizona 	 122
IL 	 121
MS - Mississippi 	 111
MA - Massachusetts 	 105
OK - Oklahoma 	 104
Tennessee 	 95
Wyo 	 94
WA - Washington 	 94
KS - Kansas 	 88
KY - Kentucky 	 85
NV - Nevada 	 83
Tenn 	 70
CA 	 69
AR - Arkansas 	 67
DE - Delaware 	 64
OR - Oregon 	 54
IA - Iowa 	 48
WV - West Virginia 	 47
N.J. 	 46
UT - Utah 	 45
MT - Montana 	 44
ND - North Dakota 	 37
DC - District of Columbia 	 34
NM - New Mexico 	 33
AK - Alaska 	 31
NE - Nebr

In [22]:
len(states)

22678

In [23]:
len(dates)

21578

In [24]:
for v in Counter(dates).most_common():
    print(v[0],'\t', v[1])

2015 	 7380
2016 	 7227
2014 	 2423
2013 	 1027
2017 	 432
2002 	 272
2005 	 271
2012 	 263
2003 	 258
2008 	 249
2001 	 244
2004 	 233
2006 	 228
2011 	 228
2000 	 221
2010 	 215
2007 	 206
2009 	 166
1969 	 5
1987 	 4
1970 	 3
1942 	 3
1989 	 3
1932 	 2
1999 	 2
1973 	 2
1984 	 1
1929 	 1
1998 	 1
1996 	 1
2026 	 1
1994 	 1
2207 	 1
3009 	 1
2018 	 1
1916 	 1
1995 	 1


### 4. Potential for profiling

In [53]:
debug=False

In [54]:
with open(doc2inc_file, 'rb') as f:
    doc2inc = pickle.load(f)

In [55]:
def unresolvable_pairs_count(participants_file_path):
    with open(participants_file_path, 'rb') as f:
        gva_data = pickle.load(f)
        
    c=0
    nc=0
    for doc_id, data in gva_data.items():
        for participant_id, participant_info in data.items():
            for doc_id2, data2 in gva_samefirstname.items():
                for participant_id2, participant_info2 in data2.items():
                    if participant_id2>participant_id and participant_info==participant_info2:
                        if debug:
                            print(doc2inc[doc_id][0], doc_id, participant_id, participant_info)
                            print(doc2inc[doc_id2][0], doc_id2, participant_id2, participant_info2)
                            print()
                        c+=1
                    elif participant_id2>participant_id:
                        nc+=1
    return c,nc

In [56]:
count, not_count=unresolvable_pairs_count(gva_input_annotations_file)
print(count, not_count)

12 296823


In [57]:
count, not_count=unresolvable_pairs_count(gva_samefirstname_annotations)
print(count, not_count)

242 296593


In [58]:
count, not_count=unresolvable_pairs_count(gva_samelastname_annotations)
print(count, not_count)

12 296823


### 5. Analysis on the entire SE data

In [1]:
se_data_folder='../../semeval18_data'
se_test_data_folder='%s/test_data' % se_data_folder
se_trial_data_folder='%s/trial_data' % se_data_folder
se_frames_file='%s/gva_frames' % se_data_folder

In [4]:
a_df = pd.read_pickle(se_frames_file)

In [9]:
len(a_df)

4580

In [17]:
name2inc=defaultdict(set)
for index, row in a_df.iterrows():
    inc_uri=row['incident_uri']
    for p in row['participants']:
        if 'Name' in p and p['Name']:
            name2inc[p['Name'].strip()].add(inc_uri)

In [21]:
for name, incs in name2inc.items():
    if len(incs)>1 and name not in ['Officer', '37', 'Deputy', 'Trooper', 'Detective']:
        print(name, len(incs))

Jonathan Martinez 2
David Anderson 2
Carl Johnson 2
Daniel Munoz 2
Wyatt Krauss 2
David Krauss 2
Charles Jackson 2
Zanyrah Taylor 2
Maria Rodriguez 2
Henry Jackson 2
Ronald Smith 2
Tyler Jordan Buchanan 2
Joel Rodriguez 2
Saeve Evans 2
Agent 2
Matthew Michael Stevenson 2
Omar Hamilton 2
Billy Williams 2
Cole Kirkpatrick 2
Sean Thomas 2
Michael Adams 2
Justin Anderson 2
Juan Trevino 2
Lawrence Anderson 2
Eric Smith 2
Malachi Hemphill 2
Allen Troyer 2
Quanisha Sims 2
Tavarius McKay 2
Vincent Wainwright 2
Demetrius Davis 2
Ronald Williams 4
Derrick Jackson 2
Ismael Moreno Jr. 2
Jordan Harris 2
Tyquone Greer 2
Mario Oliver 2
Jeremy Oliver 2
Henry Holloway 2
Marcus Oliver 2
Robert Holloway 2
Ronnie Hendrix 2
Elijah Jackson 2
Rakim Watson 3
Jorge Hernandez 2
